In [ ]:
library(ggplot2)
library(dplyr)
library(reshape2)
library(gridExtra)

In [ ]:
N <- 1000
a <- 0.015
b <- -0.09
k <- function(n=1){
        k<-runif(n,0.16,0.25)
        return(k)
        }
max_s_prop <- 0.7
age_max <- 70
age_step <- 1
N_iter=10

In [ ]:
multi_sim <- function(N,a,b,k,max_s_prop,age_max,age_step,N_iter){
    multipop<-NA
    for(replicats in 1:N_iter){
        pop <- matrix(,nrow=4,ncol=age_max)
        rownames(pop)<-c("age","Smurf_prop","n_NS","n_S")
        pop[1,]<- seq(0,age_max-1,age_step)
        cbind.na <- function(x, y){
            n <- max(length(x), length(y))
            length(x) <- n              
            length(y) <- n
            combined<-cbind(x,y)
            combined[which(is.na(combined))]<-0
            return(combined)
        }

        Smurf <- function(age){
            age_func <- a*age+b
            smurf_prop <- ifelse(age_func>0, ifelse(age_func<max_s_prop,runif(1,0,age_func*2),runif(1,0,max_s_prop)),0)
            return(smurf_prop)
        }

    pop[3,1]<-N
    for(i in pop[1,]){pop[2,i]<-Smurf(i)}
    pop[4,1]<-round(pop[3,1]*pop[2,1], 0)
    i=0
    for(j in 2:length(pop[2,])){
       i=i+1 
       pop[4,j]<-round(pop[3,i]*pop[2,j] ,0)
       pop[3,j]<-pop[3,i]-pop[4,j] 
    }
    pop<-pop[,-length(pop[1,])]
    #pop

    kill_Smurfs <- function(n_S){
        dyingSmurfs <- NA
        dyingSmurfs[1]<-round(n_S-n_S*exp(-k()),0)
        i<-1
        while(n_S>0){
            i=i+1
            n_S<-dyingSmurfs[i-1]
            dyingSmurfs[i]<-round(n_S-n_S*exp(-k()),0)
        }
        return(dyingSmurfs)
    }


    Smurf_death<-kill_Smurfs(pop[4,1])
    length(Smurf_death)<-10; Smurf_death[which(is.na(Smurf_death))]<-0
    for(i in pop[4,2:length(pop[4,])]){
        Smurf_death<-cbind.na(Smurf_death,kill_Smurfs(i))
    }
Smurf_death<-cbind(pop[4,],t(Smurf_death))
Smurf_death[is.na(Smurf_death)]<-0
Smurf_death<-as.data.frame(Smurf_death)
rownames(Smurf_death)<-pop[1,]
colnames(Smurf_death)<-seq(1:length(Smurf_death[1,]))
#Smurf_death

Smurfs_dyn<-NA
length(Smurfs_dyn)<-80
for(i in 1:length(Smurf_death[,1])){
    temp<-c(rep(NA,i-1),Smurf_death[i,])
    length(temp)<-80
    Smurfs_dyn<-rbind(Smurfs_dyn,temp)
    }
Smurfs_dyn<-as.data.frame(Smurfs_dyn[-1,])
colnames(Smurfs_dyn)<-seq(1,80,1)
rownames(Smurfs_dyn)<-pop[1,]
Smurfs_dyn[,1:80][Smurfs_dyn[,1:80] == "NULL"] <- 0
#Smurfs_dyn

Smurfs_alive<-NA
length(Smurfs_alive)<-80
for(i in 1:80){Smurfs_alive[i]=sum(as.numeric(Smurfs_dyn[,i]),na.rm=TRUE)}
pop<-rbind(pop,t(Smurfs_alive[1:69]))
rownames(pop)<-c("age","Smurf_prop","n_NS","n_S","S_alive")
#pop

Smurf_surv<-as.data.frame(apply(Smurf_death,2,sum))
Smurf_surv<-cbind(as.numeric(colnames(Smurf_death)),Smurf_surv)
rownames(Smurf_surv)<-NULL
colnames(Smurf_surv)<-c("timeAsSmurf","Smurfs_alive")
#head(Smurf_surv)
        
        multipop<-rbind(multipop,t(rbind(replicats,pop)))
        
#return(pop)
}
multipop<-as.data.frame(multipop[-1,])
multipop<-multipop%>%relocate('age')
rownames(multipop)<-NULL
return(multipop)
}

In [ ]:
sims<-suppressWarnings(multi_sim(N,a,b,k,max_s_prop,age_max,age_step, N_iter=100))

In [ ]:
#min and max proportion of Smurfs observed
min_s = 0.03
max_s = 0.086

age_max<-min(sims$age[which(sims$n_NS<1)])+5
sims<-sims[sims$age<(age_max),]
rownames(sims)<-NULL
age_prob<-sims %>% filter((S_alive/(S_alive+n_NS))>=min_s) %>% filter((S_alive/(S_alive+n_NS))<=max_s) %>% select(age)

In [ ]:
min_age<-min(as.matrix(age_prob))
median_age<-median(as.matrix(age_prob))
min_age_surv<-sims%>%filter(age==min_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%min()
median_age_surv<-sims%>%filter(age==median_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%median()

In [ ]:
#transparent_theme <- theme(
#    axis.title.x = element_blank(),
#    axis.title.y = element_blank(),
#    axis.text.x = element_blank(), 
#    axis.text.y = element_blank(),
#    axis.ticks = element_blank(),
#    panel.grid = element_blank(),
#    axis.line = element_blank(),
#    panel.background = element_rect(fill = "transparent",colour = NA),
#    plot.background = element_rect(fill = "transparent",colour = NA))

#p<-ggplot(age_prob, aes(x=age,y=age))+
#    geom_violin(draw_quantiles = c(0.25, 0.5, 0.75),trim=FALSE)+
#    ylim(0,age_max)+
#    transparent_theme+
#    coord_flip()
#p_grob<-ggplotGrob(p)

surv_plot<-ggplot(sims)+
    geom_jitter(aes(x=age,y=n_NS/N*100),size=0.1,color='dark grey', alpha=0.2)+
    geom_smooth(aes(x=age,y=n_NS/N*100),color='dark grey', linetype='dotted')+
    geom_jitter(aes(x=age,y=S_alive/N*100),size=0.1,color='deepskyblue3', alpha=0.1)+
    geom_smooth(aes(x=age,y=S_alive/N*100),color='deepskyblue3', linetype='dotted')+
    geom_smooth(aes(x=age,y=(n_NS+S_alive)/N*100),color='black')+
    #annotate("rect",xmin=min(sims[sims$S_alive>sims$n_NS,]$age), xmax=Inf, ymin=0, ymax=Inf, alpha=0.2, fill="black", label="n Smurfs > n non-Smurfs")
    annotate("rect",xmin=min_age, xmax=median_age, ymin=0, ymax=Inf, alpha=0.2, fill="red")+ 
    geom_text(aes(x=min_age, y=-3,label=min_age),size=2,size=2)+
    geom_text(aes(x=median_age, y=-3,label=median_age),size=2)+
    geom_text(aes(x=0, y=min_age_surv,label=round(min_age_surv,1)),size=2,size=2,size=2,size=2)+
    geom_text(aes(x=0, y=median_age_surv,label=round(median_age_surv,1)),size=2,size=2,size=2)+
    annotate("rect",xmin=0,xmax=median_age,ymax=min_age_surv,ymin=median_age_surv, alpha=0.2, fill="red")+
    xlim(0,age_max)+
    xlab("Age (days)")+
    ylab("Populations sizes (norm. 100)")
    #annotation_custom(grob=p_grob,xmin=0,xmax=age_max,ymin=100, ymax=200)
    
age_dist<-ggplot(age_prob, aes(x=age))+
    geom_histogram(aes(y=..density..), binwidth = 1,color='darkgray')+
    annotate("rect",xmin=min_age, xmax=median_age, ymin=0, ymax=Inf, alpha=0.2, fill="red")+
    xlim(0,age_max)+
    xlab("Age (days)")+
    ylab("Data in %S range") #+theme_minimal()

smurf_prop<-ggplot(sims,(aes(x=age,y=(S_alive/(S_alive+n_NS))*100)))+
    geom_jitter(color="deepskyblue3", size=0.01, alpha=0.1)+
    geom_smooth(color="deepskyblue3",linetype='dotted')+
    annotate("rect",xmin=min_age, xmax=median_age, ymin=0, ymax=Inf, alpha=0.2, fill="red")+
    annotate("rect",xmin=0, xmax=median_age, ymin=min_s*100, ymax=max_s*100, alpha=0.2, fill="red")+
    geom_text(aes(x=0, y=min_s*100,label=round(min_s*100,1)),size=2)+
    geom_text(aes(x=0, y=max_s*100,label=round(max_s*100,1)),size=2)+
    xlim(0,age_max)+
    ylim(0,100)+
    xlab("Age (days)")+
    ylab("% Smurfs") #+theme_minimal()
    #geom_line(aes(x=age,y=a*age+b),color='black')

suppressWarnings(grid.arrange(smurf_prop, age_dist, surv_plot, ncol=1, nrow=3, heights=c(2,2,5)))

In [ ]:
a_dependancy<-NA
for(i in seq(0.005,0.085,0.01)){
    sims<-suppressWarnings(multi_sim(N,a=i,b,k,max_s_prop,age_max,age_step, N_iter=100))
    age_prob<-sims %>% filter((S_alive/(S_alive+n_NS))>=min_s) %>% filter((S_alive/(S_alive+n_NS))<=max_s) %>% select(age)
    min_age<-min(as.matrix(age_prob))
    pop_lifespan<-sims%>%filter((n_NS+S_alive)<0.5*N)%>%select(age)%>%as.matrix()%>%mean()%>%round()
    median_age<-median(as.matrix(age_prob))
    min_age_surv<-sims%>%filter(age==min_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%min()
    median_age_surv<-sims%>%filter(age==median_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%median()
    a_dependancy<-rbind(a_dependancy, c(min_age,median_age,min_age_surv,median_age_surv,pop_lifespan))
    #print(c(min_age,median_age,min_age_surv,median_age_surv))
    }
colnames(a_dependancy)<-c('min_age','median_age','min_age_surv','median_age_surv','lifespan')
rownames(a_dependancy)<-NULL
a_dependancy<-a_dependancy[-1,]

In [ ]:
a_dependancy<-cbind(cbind("a",as.data.frame(cbind(c(a_dependancy[,1],a_dependancy[,2]), c(a_dependancy[,3],a_dependancy[,4])))),a_dependancy[,5])
colnames(a_dependancy)<-c("param","chrono","physio","lifespan")
head(a_dependancy)

In [ ]:
b_dependancy<-NA
for(i in seq(-0.09,0.09,0.01)){
    sims<-suppressWarnings(multi_sim(N,a,b=i,k,max_s_prop,age_max,age_step, N_iter=100))
    age_prob<-sims %>% filter((S_alive/(S_alive+n_NS))>=min_s) %>% filter((S_alive/(S_alive+n_NS))<=max_s) %>% select(age)
    min_age<-min(as.matrix(age_prob))
    pop_lifespan<-sims%>%filter((n_NS+S_alive)<0.5*N)%>%select(age)%>%as.matrix()%>%mean()%>%round()
    median_age<-median(as.matrix(age_prob))
    min_age_surv<-sims%>%filter(age==min_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%min()
    median_age_surv<-sims%>%filter(age==median_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%median()
    b_dependancy<-rbind(b_dependancy, c(min_age,median_age,min_age_surv,median_age_surv,pop_lifespan))
    #print(c(min_age,median_age,min_age_surv,median_age_surv))
    }
colnames(b_dependancy)<-c('min_age','median_age','min_age_surv','median_age_surv','lifespan')
rownames(b_dependancy)<-NULL
b_dependancy<-b_dependancy[-1,]

In [ ]:
b_dependancy<-cbind(cbind("b",as.data.frame(cbind(c(b_dependancy[,1],b_dependancy[,2]), c(b_dependancy[,3],b_dependancy[,4])))),b_dependancy[,5])
colnames(b_dependancy)<-c("param","chrono","physio","lifespan")
head(b_dependancy)

In [ ]:
k_dependancy<-NA
for(i in seq(0.1,0.9,0.1)){
    sims<-suppressWarnings(multi_sim(N,a,b,k=i,max_s_prop,age_max,age_step, N_iter=100))
    age_prob<-sims %>% filter((S_alive/(S_alive+n_NS))>=min_s) %>% filter((S_alive/(S_alive+n_NS))<=max_s) %>% select(age)
    min_age<-min(as.matrix(age_prob))
    median_age<-median(as.matrix(age_prob))
    pop_lifespan<-sims%>%filter((n_NS+S_alive)<0.5*N)%>%select(age)%>%as.matrix()%>%mean()%>%round()
    min_age_surv<-sims%>%filter(age==min_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%min()
    median_age_surv<-sims%>%filter(age==median_age)%>%mutate((S_alive+n_NS)/N*100,.keep="none")%>%as.matrix()%>%median()
    k_dependancy<-rbind(k_dependancy, c(min_age,median_age,min_age_surv,median_age_surv,pop_lifespan))
    #print(c(min_age,median_age,min_age_surv,median_age_surv))
    }
colnames(k_dependancy)<-c('min_age','median_age','min_age_surv','median_age_surv','lifespan')
rownames(k_dependancy)<-NULL
k_dependancy<-k_dependancy[-1,]

In [ ]:
k_dependancy<-cbind(cbind("k",as.data.frame(cbind(c(k_dependancy[,1],k_dependancy[,2]), c(k_dependancy[,3],k_dependancy[,4])))),k_dependancy[,5])
colnames(k_dependancy)<-c("param","chrono","physio","lifespan")
head(k_dependancy)

In [ ]:
test<-as.data.frame(rbind(a_dependancy,b_dependancy,k_dependancy))

ggplot(test,aes(x=chrono/lifespan,y=physio/100,color=param))+
    geom_jitter()+
    geom_density_2d(binwidth=0.01)